# Grove LED Bar Example

This example shows how to use the [Grove LED Bar](http://www.seeedstudio.com/depot/Grove-LED-Bar-v20-p-2474.html) on the board.  The LED bar has 10 LEDs: 8 green LEDs, 1 orange LED, and 1 red LED. The brightness for each LED can be set independantly.

For this notebook, a PYNQ Arduino shield is also required.  The LED bar is attached to the G4 connection on the shield. The grove LED bar also works with PMODA and PMODB on the board. 

In [1]:
# Make sure the base overlay is loaded
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

### 1. Instantiate and reset LED Bar

In [2]:
from pynq.lib.arduino import Grove_LEDbar
from pynq.lib.arduino import ARDUINO_GROVE_G4

# Instantiate Grove LED Bar on Arduino shield G4
ledbar = Grove_LEDbar(base.ARDUINO,ARDUINO_GROVE_G4)
ledbar.reset()

### 2. Turn individual LEDs on or off

Write a 10-bit binary pattern, with each bit representing the corresponding LED. 1 = on, 0 = off

In [3]:
from time import sleep

# Light up different bars in a loop
for i in range(2):
    ledbar.write_binary(0b1010100000)
    sleep(0.5)
    ledbar.write_binary(0b0000100100)
    sleep(0.5)
    ledbar.write_binary(0b1010101110)
    sleep(0.5)
    ledbar.write_binary(0b1111111110)
    sleep(0.5)

### 3. Set LEDs individually with different brightness levels

The brightness of each LED can be set individually by writing a list of 10x 8-bit values to the LED bar. 0 is off, 0xff is full brightness. 

In [4]:
# Brightness 0-255
HIGH = 0xFF
MED  = 0xAA
LOW  = 0x01
OFF  = 0X00

brightness = [OFF, OFF, OFF, LOW, LOW, MED, MED, HIGH, HIGH, HIGH]

ledbar.write_brightness(0b1111111111,brightness)

### 4.  Set the "level" or the number of LEDs which are set 

A number or level of LEDs can be turned on, started from either end of the LED bar. For example, this feature could be used to indicate the level of something being measured.

write_level(level, bright_level, green_to_red)

* level is the number of LEDs that are on.
* bright_level [0-10] is the level of brightness
* green_to_red = 1 means the LEDs start being lit from the "green" end of the LED bar
* green_to_red = 0 means the LEDs start being lit from the "red" end of the LED bar.

For example, ledbar.write_level(5,4,1) will light 5 LEDs, to brightness 4 (out of 10) and will start from the Green LED (the LED furthest away from Grove connector on the LED bar module.)


In [5]:
for i in range (1,11):
    ledbar.write_level(i,3,0)
    sleep(0.3)
for i in range (1,10):
    ledbar.write_level(i,3,1)
    sleep(0.3)    

### 5. Controlling the LED Bar from the board buttons

This cell demonstrates controlling the "level" of the LEDs from onboard buttons. 

* Button 0 to increase level
* Button 1 to decrease level
* Button 3 to exit

In [6]:
btns = [base.buttons[index] for index in range(4)] 
i = 1
ledbar.reset()

done = False
while not done:
    if (btns[0].read()==1):
        sleep(0.2)
        ledbar.write_level(i,2,1)
        i = min(i+1,9)
    elif (btns[1].read()==1):
        sleep(0.2)
        i = max(i-1,0)
        ledbar.write_level(i,2,1)
    elif (btns[3].read()==1):
        ledbar.reset()
        done = True